Evaluating the error individually and with the difference from sharpening GOES images from bands 7 and 14 using laplace and unsharpmask sharpening.

In [1]:
import os
import random

import numpy as np
import pandas as pd

from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

random.seed(42)

In [2]:
def control(path, file):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    mae = mean_absolute_error(img, control)
    rmse = mean_squared_error(img, control, squared=False)
    return {'File': file, 'Band': file[19:21], 'Control_MAE': mae, 'Control_RMSE': rmse}

def laplace_sharpening(path, file):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + laplace_edges
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'File': file, 'Band': file[19:21], 'LP_MAE': mae, 'LP_RMSE': rmse}

def unsharpmask_sharpening(path, file):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    img = np.load(path + file)
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    mae = mean_absolute_error(img, sharpimg)
    rmse = mean_squared_error(img, sharpimg, squared=False)
    return {'File': file, 'Band': file[19:21], 'UM_MAE': mae, 'UM_RMSE': rmse}

In [3]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    prefix = file[:19] + band + file[21:41]
    for x in os.listdir(path):
        if x.startswith(prefix):
            return x

def update_dict(path, file, data):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    newdata = {}
    newdata.update(control(path, file))
    newdata.update(laplace_sharpening(path, file))
    newdata.update(unsharpmask_sharpening(path, file))
    data.update(newdata)

## Evaluating Error

In [26]:
path = '../../GOES_Files/npy_files/'
data = []

for i in range (20):
    file14 = None
    while file14 == None:
        file07 = ''
        while file07[19:21] != '07':
            file07 = random.choice(os.listdir(path))
        file14 = find_npy_file(path, file07, '14')
    
    data07 = {}
    update_dict(path, file07, data07)
    
    data14 = {}
    update_dict(path, file14, data14)

    datadiff = {}
    diffpath = '../../GOES_Files/diff_files/'
    diffname = file07[:19] + 'diff' + file07[21:]
    np.save(diffpath + diffname, np.load(path + file07) - np.load(path + file14))
    update_dict(diffpath, diffname, datadiff)

    data.append(data07)
    data.append(data14)
    data.append(datadiff)
    
df = pd.DataFrame(data)

In [27]:
print("Mean Error")
df.groupby(df['Band']).agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 
                            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 
                            'UM_MAE': 'mean', 'UM_RMSE': 'mean'})

Mean Error


,Control_MAE,Control_RMSE,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE
Band,,,,,,
07,0.694774,1.028727,0.561817,0.850620,0.573752,0.859156
14,0.846677,1.320872,0.687813,1.100870,0.703626,1.114825
di,0.549977,0.885623,0.455815,0.749065,0.468358,0.762672


In [28]:
print("Standard Deviation of Error")
df.groupby(df['Band']).agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 
                            'LP_MAE': 'std', 'LP_RMSE': 'std', 
                            'UM_MAE': 'std', 'UM_RMSE': 'std'})

Standard Deviation of Error


,Control_MAE,Control_RMSE,LP_MAE,LP_RMSE,UM_MAE,UM_RMSE
Band,,,,,,
07,0.194978,0.321476,0.155565,0.262724,0.156609,0.263128
14,0.340440,0.553929,0.272891,0.461252,0.278271,0.467186
di,0.301239,0.489006,0.244338,0.413313,0.251370,0.420657


&nbsp;

Differences

In [29]:
cols = ['Control_MAE', 'Control_RMSE', 'LP_MAE', 'LP_RMSE', 'UM_MAE', 'UM_RMSE']
diffdf = pd.DataFrame()

for name in cols:
    for07 = df[df['Band']=='07'][name].tolist()
    for07 = np.asarray(for07)

    for14 = df[df['Band']=='14'][name].tolist()
    for14 = np.asarray(for14)

    diff = for07-for14
    diff = diff.tolist()
    diffdf[name] = diff

In [30]:
print("Difference of Mean Error")
diffdf.agg({'Control_MAE': 'mean', 'Control_RMSE': 'mean', 
            'LP_MAE': 'mean', 'LP_RMSE': 'mean', 
            'UM_MAE': 'mean', 'UM_RMSE': 'mean'})

Difference of Mean Error


Control_MAE    -0.151903
Control_RMSE   -0.292145
LP_MAE         -0.125996
LP_RMSE        -0.250250
UM_MAE         -0.129874
UM_RMSE        -0.255669
dtype: float64

In [31]:
print("Difference of Standard Deviation of Error")
diffdf.agg({'Control_MAE': 'std', 'Control_RMSE': 'std', 
            'LP_MAE': 'std', 'LP_RMSE': 'std', 
            'UM_MAE': 'std', 'UM_RMSE': 'std'})

Difference of Standard Deviation of Error


Control_MAE     0.182462
Control_RMSE    0.299595
LP_MAE          0.151081
LP_RMSE         0.262794
UM_MAE          0.155803
UM_RMSE         0.267724
dtype: float64